In [4]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# import data_toolbox
def normal_heartrate(num):
    """
    Function to normalize heart rate values.

    Parameters:
        num: the originial input value
    Return:
        num: the normalized output value
    """
    # Return null values direcly
    if pd.isna(num):
        return num
    # Remove values out of range
    elif num > 300 or num < 0:
        return np.nan
    # Return normal values directly
    else:
        return num
# change to your folder path
# os.chdir('/Users/xiao-zy19/Desktop/Johns Hopkins/Biomedical Data Design/EICU Database/eicu-collaborative-research-database-demo-2.0.1') 
# os.chdir('/home/en580-zxia028/EICU/data')
os.chdir('C:\JHU\BDD\ICU streaming data\eicu-collaborative-research-database-demo-2.0.1\demo')

# Directly use the final_available_patients list
patient_id= pd.read_csv('Final_available_patients.csv')
patient_id_all = patient_id['patientunitstayid'].unique()
patient_id_all.sort()
print(patient_id_all)

[ 151900  152954  187150  197921  202294  209139  222408  225206  242895
  243097  249328  250719  250724  260468  261520  264276  276594  276815
  284517  291628  301185  302668  304256  312258  341780  343093  379119
  383358  384028  384485  389703  408514  426976  443664  447543  454584
  468560  472359  483209  486824  490454  490809  498848  503222  506770
  520522  521456  564216  585971  640796  688938  701925  756295  851094
  859201  913057  931649  939222  960943  963136  965083  965562  966066
  968367  968871  977218 1036759 1056912 1059533 1060144 1061585 1062496
 1068646 1072883 1090567 1096127 1113145 1118668 1131143 1252888 1334350
 1363755 1441610 1455088 1474690 1532382 1559197 1567943 1568559 1571533
 1588817 1588896 1602040 1692067 1735220 1749391 1751224 1774133 1775944
 1811733 1824650 1837638 1837964 2075529 2114686 2154422 2283009 2284508
 2419768 2558122]


In [5]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import argparse
import time
from utils import patient_id_age, heart_rate, align_data,create_index, blood_pressure

In [6]:
#start_time_all = time.time()

# os.chdir('/Users/xiao-zy19/Desktop/Johns Hopkins/Biomedical Data Design/EICU Database/eicu-collaborative-research-database-demo-2.0.1') 
# os.chdir('/home/en580-zxia028/EICU_demo/data')
os.chdir('C:\JHU\BDD\ICU streaming data\eicu-collaborative-research-database-demo-2.0.1\demo')
folder_path = 'output/hr'
# os.chdir(r'C:\Users\xiao-zy19\Desktop\Johns Hopkins\Biomedical Data Design\EICU\eicu-collaborative-research-database-2.0')

# get patient id
_, patient_age, patient_offset = patient_id_age()

# add parser
#parser = argparse.ArgumentParser(description='''
    #Process EICU data in batches. 
    #This python script allows you to process data from the EICU database in specified batch sizes, 
    #or all at once.
#''')
#parser.add_argument('--batch', type=int, default=0, help='batch number to process')
#parser.add_argument('--batch_size', default='all', help='size of each batch or "all"')

#args = parser.parse_args()

batch = 0
batch_size = 'all'

# check if batch_size is 'all'
if batch_size.lower() == 'all':
    batch_size = len(patient_id_all)  # set batch_size to be the number of patients
    patient_batch = [patient_id_all[i:i + batch_size] for i in range(0, len(patient_id_all), batch_size)]
    # patient_batch = patient_id
    file_name = 'HR_all.csv'
    print('batch size: ', batch_size)
    print('now processing all data')
else:
    batch_size = int(batch_size)  # turn batch_size into integer
    patient_batch = [patient_id_all[i:i + batch_size] for i in range(0, len(patient_id_all), batch_size)]
    file_name = 'HR_full_' + str(batch) + '.csv'
    print('batch size: ', batch_size)
    print('now processing batch: ', batch, '/', len(patient_batch)-1)

print(patient_batch[batch].shape)
# extract heart rate data
_,_,_,_,_,_,_,_,BP, _ = blood_pressure(patient_batch[batch])
#data_full, data_full_index = align_data(patient_batch[batch], patient_offset, HR, graph=False)

# folder_path = '/home/en580-zxia028/output/hr/data_full'
# file_name = 'HR_full_' + str(batch) + '.csv'
#data_full.to_csv(os.path.join(folder_path, file_name), index=False)

#end_time_all = time.time()
#print("Execution time: ", end_time_all - start_time_all, " seconds")
print(BP.head())

batch size:  110
now processing all data
(110,)
Loading Blood Pressure Data...
Blood Pressure Data Loaded. Time: 1.77s
        patientunitstayid  observationoffset    BP
168549             151900               -238  77.0
168550             151900               -223  78.0
168551             151900               -213  77.0
168552             151900               -193  83.0
168553             151900               -183  79.0


In [7]:
data=BP.copy()
column_names = data.columns.tolist()
print(f"column names: {column_names}")

    # select the wanted patient
data = data[data[column_names[0]].isin(patient_batch[batch])]
#print(data)
patient_offset = patient_offset[patient_offset[column_names[0]].isin(patient_batch[batch])]

    # transform patient_offset to hours
patient_hours = patient_offset.copy().reset_index(drop=True)
#print(patient_hours.head())
patient_hours["unitdischargeoffset"] = np.floor(
    patient_hours["unitdischargeoffset"] / 60
).astype(int)

    # get unique patient ids in data
unique_data_patient_ids = data["patientunitstayid"].unique()
patient_hours = patient_hours[
    patient_hours["patientunitstayid"].isin(unique_data_patient_ids)
].reset_index(drop=True)

    # change observationoffset to hours
data_hour_buf = data.copy().reset_index(drop=True)
    # data_hour_buf["observationoffset"] = np.floor(data_hour_buf["observationoffset"]/60).astype(int)
data_hour_buf[column_names[1]] = np.floor(
    data_hour_buf[column_names[1]] / 60
).astype(int)
data_hour_buf_mean = data_hour_buf.groupby(
    [column_names[0], column_names[1]], as_index=False
)[column_names[2]].mean().copy()
#计算数值列的最大值
data_hour_buf_max = data_hour_buf.groupby([column_names[0], column_names[1]], as_index=False)[column_names[2]].max()
data_hour_buf_max = data_hour_buf_max.rename(columns={column_names[2]: "BP_max"})
# 计算数值列的最小值
data_hour_buf_min = data_hour_buf.groupby([column_names[0], column_names[1]], as_index=False)[column_names[2]].min()
data_hour_buf_min = data_hour_buf_min.rename(columns={column_names[2]: "BP_min"})
# 计算数值列的方差
data_hour_buf_var = data_hour_buf.groupby([column_names[0], column_names[1]], as_index=False)[column_names[2]].var()
data_hour_buf_var = data_hour_buf_var.rename(columns={column_names[2]: "BP_var"})
data_hour_buf_var['BP_var'] = data_hour_buf_var['BP_var'].fillna(0)
print(patient_hours.head())
print(data_hour_buf_max.head())
#data_hour_buf.sort_values(by=[column_names[0], column_names[1]], inplace=True)

column names: ['patientunitstayid', 'observationoffset', 'BP']
   patientunitstayid  unitdischargeoffset
0             151900                   57
1             152954                   70
2             187150                  160
3             197921                   91
4             202294                   87
   patientunitstayid  observationoffset  heartrate_max
0             151900                 -4           83.0
1             151900                 -3           89.0
2             151900                 -2          134.0
3             151900                 -1          118.0
4             151900                  0          121.0


In [8]:
data_hour_cleaned = pd.merge(
    data_hour_buf_mean,data_hour_buf_max, on=[column_names[0], column_names[1]], how="left"
)
data_hour_cleaned = pd.merge(
    data_hour_cleaned,data_hour_buf_min, on=[column_names[0], column_names[1]], how="left"
)
data_hour_cleaned = pd.merge(
    data_hour_cleaned,data_hour_buf_var, on=[column_names[0], column_names[1]], how="left"
)
data_hour_cleaned = pd.merge(
    data_hour_cleaned,patient_hours, on=column_names[0], how="left"
)

data_hour_cleaned = data_hour_cleaned[
    data_hour_cleaned[column_names[1]] <= data_hour_cleaned["unitdischargeoffset"]
]
    # data_hour = data_hour_cleaned.drop(["unitdischargeoffset"], axis=1)
    # print(data_hour)
data_hour = data_hour_cleaned.copy()
print(data_hour.head())
data_hour.to_csv('BP_final.csv', index=False)

   patientunitstayid  observationoffset          BP  heartrate_max  \
0             151900                 -4   78.800000           83.0   
1             151900                 -3   83.666667           89.0   
2             151900                 -2  100.666667          134.0   
3             151900                 -1   94.500000          118.0   
4             151900                  0   88.461538          121.0   

   heartrate_min  heartrate_var  unitdischargeoffset  
0           77.0       6.200000                   57  
1           80.0      22.333333                   57  
2           75.0     530.266667                   57  
3           71.0    1104.500000                   57  
4           66.0     279.935897                   57  
